In [ ]:
# if you are using this script on server you can specify th visible free cuda device
# the memory usage of devices and their id can be checked with 'nvidia-smi' or 'nvidia-cdl' commands 

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2' # here the id of the device

### Cloning repo from git


In [ ]:
!git clone https://Rookie20001122@github.com/skoltech-nlp/paradetox.git

Cloning into 'paradetox'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 87 (delta 32), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (87/87), 1.01 MiB | 2.38 MiB/s, done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'style_transfer'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['detoxification_metrics.ipynb', 'sample_data.tsv', 'checkpoints', 'model_saved', '__pycache__', 'data', 'output', 'network_unsupervised.py', 'process_dataset.py', 'supervised_test.py', 'process_nonparallel_dataset.py', 'network.py', 'supervised_output_mid24.txt', 'output_test', 'supervised_train.py', 'Untitled0.ipynb']


### Installing libraries

In [ ]:
!pip install transformers
!pip install python-box
!pip install flair
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 54.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━

### Preparation

In [ ]:
from box import Box

In [ ]:
args = Box({
    'batch_size': 32,
    'cola_classifier_path': '/content/drive/MyDrive/style_transfer',
    'wieting_tokenizer_path': 'sim.sp.30k.model',
    'wieting_model_path': 'sim.pt',
    't1': 75., # this is default value
    't2': 70., # this is default value
    't3': 12. # this is default value
})

In [ ]:
preds = []
inputs = []
with open("/content/drive/MyDrive/style_transfer/output_test/supervised_output_mid12.txt", "r") as fp:
    line = fp.readline()
    while line:
      content = line.split("<EOS>")
      for i in range(len(content)):
        if i == 0:
          preds.append(content[i].strip().replace("<PAD>", ""))
        if i == 1:
          inputs.append(content[i].strip().replace("<PAD>", "")) 
      line = fp.readline()       

### Style Transfer Accuracy metric

In [ ]:
from paradetox.evaluation_detox.metric_tools.style_transfer_accuracy import classify_preds
import numpy as np

In [ ]:
accuracy_by_sent = classify_preds(args, inputs)
accuracy = np.mean(accuracy_by_sent)
print('\n')
print(accuracy)

Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:00<00:00,  2.48it/s]



1.0


### Content Similarity metrics

In [ ]:
!wget https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
!unzip wieting_similarity_data.zip

--2023-04-13 03:08:31--  https://storage.yandexcloud.net/nlp/wieting_similarity_data.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74066204 (71M) [application/x-zip-compressed]
Saving to: ‘wieting_similarity_data.zip’

wieting_similarity_ 100%[===================>]  70.63M  43.8MB/s    in 1.6s    

2023-04-13 03:08:33 (43.8 MB/s) - ‘wieting_similarity_data.zip’ saved [74066204/74066204]

Archive:  wieting_similarity_data.zip
  inflating: sim.pt                  
  inflating: sim.sp.30k.model        


In [ ]:
from paradetox.evaluation_detox.metric_tools.content_similarity import wieting_sim
similarity_by_sent = wieting_sim(args, inputs, preds)
avg_sim_by_sent = similarity_by_sent.mean()
print('\n')
print(avg_sim_by_sent)

Calculating similarity by Wieting subword-embedding SIM model


100%|██████████| 1/1 [00:00<00:00, 15.89it/s]



0.6545021448816571


### Fluency metrics

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("jxuhf/roberta-base-finetuned-cola")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("jxuhf/roberta-base-finetuned-cola")

In [ ]:
import torch
tokens = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)
logits = outputs.logits
l = torch.nn.Softmax(dim=1)
cola_score = torch.mean(l(logits)[:, 1])
print(cola_score)

tensor(0.5799, grad_fn=<MeanBackward0>)
